# Immersed Cylinder benchmarking

This notebook will contain all the required scripts and commands to produce the
benchmarking for the immersed cylinder case.

We expect the simulation for the `cylinder_benchmark` example have been
completed and that the resulting data is available in the `cylinder_benchmark`
subdirectory of the results folder.

The benchmarking consists of a few different sub-benchmarks:

1. Inflection point detection.
2. Backdraft region detection.
3. Leakage detection.


![image](media/close_up.png)

In [ ]:
# Assign flags for debugging to alter the behaviour of the program in addition
# to the list of experiments to compare.

debug = False
root_folder = "../../"
data_folder = "results/hpc/cylinder_benchmark"

# Define the list of data files and the labels to give them
experiments = [
    "meshed",
    "explicit_b_100",
    "explicit_b_200",
    "explicit_b_400",
    "explicit_b_800",
    "explicit_b_1600",
    "explicit_b_3200",
]
labels = [
    "Baseline",
    "Explicit 100",
    "Explicit 200",
    "Explicit 400",
    "Explicit 800",
    "Explicit 1600",
    "Explicit 3200",
]

In [ ]:
# Setup the paths for the project and load the external modules

# Load the external modules
import sys
import os

try:
    import import_ipynb
    import numpy as np
except:
    print("import_ipynb not installed. Installing it now.")
    %pip install import_ipynb numpy
    import import_ipynb
    import numpy as np

# Load internal modules
try:
    from nekotop import *
except:
    sys.path.append(root_folder + "scripts/modules")
    from nekotop import *

# If the data folder is not an absolute path, make it relative to the root
if data_folder[0] != "/":
    data_folder = os.path.realpath(os.path.join(root_folder, data_folder))

# Check that the folder exists
if not os.path.exists(data_folder):
    raise FileNotFoundError(
        "The path to the results folder does not exist.\n"
        "Please run the benchmark first.\n"
        "\tcd NEKO_TOP_ROOT\n"
        "\t./setup.sh\n"
        "\t./run.sh cylinder_benchmark\n"
    )

def no_data(ax):
    ax.text(s="No data",
            x=0.5,
            y=0.5,
            fontsize=24,
            color="red",
            transform=ax.transAxes,
            ha="center",
            va="center")

figure_style_2d = {
    "facecolor": [148.0 / 256.0, 163.0 / 256.0, 179.0 / 256.0, 0.5],
    # "constrained_layout": True,
}

# Inflection point detection

Separation points are detected by looking for inflection points in the flow
field around the cylinder. The inflection points are detected by looking for
zero crossings in the velocity component tangential to the cylinder surface.

The angle of the separation point is then calculated by looking at the angle
between the separation point and x axis. This angle provides a time series as
vortices are build and shedd. These time series will be almost impossible to
line up between the meshed cylinder and the immersed boundary cylinder, but the
overall trend should be similar.

We will be computing a number of statistical measures to compare.

- The mean angle of the separation point, or the bias of the signal.
- The amplitude of the signal.
- The dominant frequency of the signal.
- The Strouhal number for the flow.

We expect to see the signal to be separated into 3 parts, the build up of the
boundary layer, build-up of the vortex and the shedding of the vortex. All the
measures are computed for the stable phase shedding of vortices.


## Python function

The inflection benchmarking have been implemented in the inflection point
detection [notebook](inflection_points.ipynb). The notebook contains the
implementation of the inflection point detection and the statistical measures
to compare the results. Additionally the notebook contain notes and comments on
the implementation.

The benchmark should be executed as follows:

```python
import import_ipynb
from inflection_points import inflection_benchmark

bench = inflection_benchmark("path/to/inflection.csv")
```

The returned object `bench` is a dictionary containing the statistical measures
of the inflection point detection. As well as the computed angles and
timestamps.


In [ ]:
# Import the inflection benchmark and run it for all the data files
#
# This cell will import and run the inflection point benchmarking for all the
# data files in the folder. The benchmark will compare the data to the baseline
# and produce a plot of the inflection points for each case.

from inflection_points import inflection_benchmark

# Run the benchmark
if "inflection" not in locals() or debug:
    inflection = [None for _ in range(len(experiments))]
    inflection_labels = [None for _ in range(len(experiments))]
    for i in range(len(experiments)):
        file = os.path.join(data_folder, experiments[i], "inflection.csv")
        try:
            inflection[i] = inflection_benchmark(file)
            inflection_labels[i] = labels[i]
        except:
            print("Failed to load: ", experiments[i])

    # Remove the failed experiments
    inflection_labels = [x for x in inflection_labels if x is not None]

In [ ]:
# Plot the inflection point angles for each case
import matplotlib.pyplot as plt

max_columns = 3

rows = len(experiments) // max_columns + 1
columns = min(len(experiments), max_columns)

fig = plt.figure(figsize=(6 * columns, 5 * rows))
fig.set(**figure_style_2d)

gs = fig.add_gridspec(rows, columns, hspace=0, wspace=0)
ax = gs.subplots(sharex=True, sharey=True)

(x0, x1, y0, y1) = (float("inf"), float("-inf"), float("inf"), float("-inf"))

for i in range(len(experiments)):
    idx = (i // max_columns, i % max_columns)

    # Plot the inflection angles
    if inflection[i] is not None:
        ax[idx].plot(inflection[i]["times"], inflection[i]["angles"])

        # Add vertical lines to show the building and layer formation times
        ax[idx].axvline(
            inflection[i]["times"][inflection[i]["i_building"]],
            color="red",
            linestyle="--",
            label="Vortex build-up time",
        )
        ax[idx].axvline(
            inflection[i]["times"][inflection[i]["i_boundary"]],
            color="green",
            linestyle="--",
            label="Boundary formation time",
        )
    else:
        no_data(ax[idx])

    ax[idx].set_title(labels[i], x=0.5, y=0.9, fontsize=16)
    x0 = min(x0, ax[idx].axis()[0])
    x1 = max(x1, ax[idx].axis()[1])
    y0 = min(y0, ax[idx].axis()[2])
    y1 = max(y1, ax[idx].axis()[3])

for idx in np.ndindex(rows, columns):
    ax[idx].grid(True, which="both", linestyle="--", linewidth=0.5)

    ax[idx].axis([x0, x1, 1.1 * y0, 1.1 * y1])

    if idx[0] == rows - 1:
        ax[idx].set_xlabel("Time (s)")
    if idx[1] == 0:
        ax[idx].set_ylabel("Angle (rad)")

In [ ]:
# Definition of a function used to plot a table of information based on the
# inflection data


def plot_table(data: np.ndarray, column_labels, row_labels, precision=3):
    """
    Plot a table of data with the given column labels and row labels.

    Parameters
    ----------
    data : np.ndarray
        The data to plot in the table.
    column_labels : list
        The labels for the columns.
    labels : list
        The labels for the rows.
    precision : int
        The number of decimal places to display.
    """

    # Preallocate the data text
    data_text = [["" for _ in range(len(column_labels))]
                 for _ in range(len(row_labels))]

    # Fill the data text with the values
    for (i, j) in [(i, j) for i in range(len(row_labels))
                   for j in range(len(column_labels))]:
        data_text[i][j] = f"{data[i,j]:.{precision}f}"

    fig, ax = plt.subplots(figsize=(len(column_labels) * 2.0,
                                    len(row_labels) * 0.5))

    fig.set(**figure_style_2d)

    ax.axis("off")
    table = ax.table(
        cellText=data_text,
        colLabels=column_labels,
        rowLabels=row_labels,
        loc="center",
        cellLoc="center",
        colLoc="center",
        bbox=[0, 0, 1, 1],
        edges="horizontal",
    )
    table.auto_set_font_size(False)
    table.set_fontsize(14)
    table.scale(1, 1.5)

## Bias of the signal

The bias of the signal is calculated as the mean angle of the separation point.
An angle of 0 rad is aligned with the x-axis.
In addition tto the bias for each inflection point, we also compute the mean
difference from the meshed cylinder to the immersed boundary cylinder.

In [ ]:
# Print the bias of the inflection points

column_labels = ["Top", "Center", "Bottom", "Mean difference"]

bias = np.asarray([i["bias"] for i in inflection if i is not None])
bias = np.concatenate(
    (bias, np.mean(bias - bias[0, :], axis=1).reshape(-1, 1)), axis=1)

plot_table(bias, column_labels, inflection_labels)

## Dominant frequencies

In [ ]:
# Print the dominant frequency of the inflection points

column_labels = ["Top", "Center", "Bottom", "Mean difference"]

freq = np.asarray([i["max_freq"] for i in inflection if i is not None])
freq = np.concatenate(
    (freq, np.mean(freq - freq[0, :], axis=1).reshape(-1, 1)), axis=1)

plot_table(freq, column_labels, inflection_labels)

## Amplitude of the signal

In [ ]:
# Print the amplitude of the inflection points movement

column_labels = ["Top", "Center", "Bottom", "Mean difference"]

amplitude = np.asarray([i["amplitude"] for i in inflection if i is not None])
amplitude = np.concatenate(
    (amplitude, np.mean(amplitude - amplitude[0, :], axis=1).reshape(-1, 1)),
    axis=1)

plot_table(amplitude, column_labels, inflection_labels)

In [ ]:
# Print the Strouhal number for each case
#
# The Strouhal number is a dimensionless number that describes the oscillation
# of a body in a fluid. It is defined as:
#
#     St = f * D / U
#
# where:
#     f: frequency of oscillation
#     D: diameter of the body
#     U: ambient velocity of the fluid
#
# The Strouhal number is a useful number to compare the oscillation of bodies
# of different sizes and in different fluids. It is also useful to compare the
# oscillation of bodies in different flow regimes.
#
# We will compute the Strouhal number for each inflection point in each case and
# print the mean value of the Strouhal number for each case.


def compute_strouhal(frequencies, diameter, ambient_velocity):
    return np.asarray([f * diameter / ambient_velocity for f in frequencies])


strouhal = np.asarray([
    compute_strouhal(i["max_freq"], 1.0, 1.0).mean()
    for i in inflection if i is not None])

plot_table(strouhal.reshape(-1, 1), ["Strouhal number"], inflection_labels)

## Timing for the phases

The flow goes through a few different phases during the simulation. First the
boundary layers are formed, then the vortices are formed and finally the
vortices are shed. In general only the shedding of the vortices are of interest
for the Strouhal number. But for the immersed boundary method, the boundary
layer formation do tell us something about the leakage.

In [ ]:
# Print the boundary layer building time

column_labels = ["Layer", "Build-up"]

times = np.asarray((
    [i["times"][i["i_boundary"]] for i in inflection if i is not None],
    [i["times"][i["i_building"]] for i in inflection if i is not None]
)).transpose()

plot_table(times, column_labels, inflection_labels)

# Leakage detection

The leakage detection is done by looking at the velocity component inside the
cylinder. The velocity component should be zero inside the cylinder, but due to
the immersed boundary method, there will be some leakage. The leakage is
calculated as the mean velocity inside the cylinder.
Additionally we also report the time wise standard deviation of the velocity, max 
and min velocity inside the cylinder.

In [ ]:
from leakage import leakage_benchmark

# Run the benchmark
if "leakage" not in locals() or debug:
    leakage = [None for _ in range(len(experiments))]
    leakage_labels = [None for _ in range(len(experiments))]
    for i in range(0, len(experiments)):
        file = os.path.join(data_folder, experiments[i], "leakage.csv")
        try:
            leakage[i] = leakage_benchmark(file)
            leakage_labels[i] = labels[i]
        except:
            print("Failed to load: ", experiments[i])
            pass

In [ ]:
# Draw the leakage plot

max_columns = 3

rows = len(experiments) // max_columns + 1
columns = min(len(experiments), max_columns)

fig = plt.figure(figsize=(6 * columns, 5 * rows))
fig.set(**figure_style_2d)

gs = fig.add_gridspec(rows, columns, hspace=0, wspace=0)
ax = gs.subplots(sharex=True, sharey=True)

# Ensure that ax is a list
if not isinstance(ax, np.ndarray):
    ax = np.asarray([[ax]])

(x0, x1, y0, y1) = (float("inf"), float("-inf"), float("inf"), float("-inf"))

for i in range(len(experiments)):
    idx = (i // max_columns, i % max_columns)

    # Plot the inflection angles
    if leakage[i] is not None:
        ax[idx].plot(leakage[i]["times"], leakage[i]["mean"], label="Mean")
        ax[idx].plot(leakage[i]["times"], leakage[i]["max"], label="Max")
        ax[idx].plot(leakage[i]["times"], leakage[i]["min"], label="Min")
    else:
        no_data(ax[idx])

    ax[idx].set_title(labels[i], x=0.5, y=0.9, fontsize=16)
    ax[idx].set_yscale("log")
    ax[idx].grid(True, which="both", linestyle="--", linewidth=0.5)

    x0 = min(x0, ax[idx].axis()[0])
    x1 = max(x1, ax[idx].axis()[1])
    y0 = min(y0, ax[idx].axis()[2])
    y1 = max(y1, ax[idx].axis()[3])

for idx in np.ndindex(rows, columns):

    ax[idx].axis([x0, x1, 1.1 * y0, 1.1 * y1])

    if idx[0] == rows - 1:
        ax[idx].set_xlabel("Time (s)")
    if idx[1] == 0:
        ax[idx].set_ylabel("Average velocity (m/s)")

In [ ]:
# Plot the smallest maximum velocity for as a function of Brinkman number

fig = plt.figure(figsize=(6, 5))
fig.set(**figure_style_2d)

ax = fig.add_subplot(111)

# Compute the Brinkman number
brinkman = []
max_vel = []
for i in range(len(experiments)):
    if leakage[i] is None:
        continue

    id = experiments[i].find("b_")
    if id != -1:
        brinkman_str = experiments[i][id+2:]
        max_vel.append(leakage[i]["max"].min())
        brinkman.append(float(brinkman_str))

if len(brinkman) > 0:
    ax.plot(brinkman, max_vel, marker="o", linestyle="--")
else:
    no_data(ax)

ax.set_xlabel("Brinkman number")
ax.set_ylabel("Minimum maximum velocity (m/s)")
ax.set_title("Minimum maximum velocity as a function of Brinkman number")
ax.grid(True, which="both", linestyle="--", linewidth=0.5)

# The end

In [ ]:
%%html

<!-- Setup of the slide show style -->
<style>
    /* Set the background color for the exported HTML slide show */
    .reveal {
        background-image: url('./media/website-background-1.png');
        background-size: cover;
    }
</style>